In [1]:
from spp_net_f import *
from spp_layer_f import *
from load_mnist import *
import os
import time
import input_data
import tensorflow as tf
import numpy as np



train_size = 3060
batch_size = 50
max_epochs =10
num_class = 10
eval_frequency = 100
max_steps = 100000

     



/home/user001/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
class data_label_data():
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def next_batch(self, batch_size):
        label_len = len(self.label)
        shuffle_index = np.arange(label_len)
        np.random.shuffle(shuffle_index)
        batch_index = shuffle_index[:batch_size]
        data_batch = self.data[batch_index]
        label_batch = self.label[batch_index]
        return [data_batch, label_batch]
class load_data(data_label_data):
    def __init__(self, train, test):
        self.train = train
        self.test = test

def train():
    global_step = tf.Variable(0, trainable=False)
    spp_net = SPPnet()
    spp_net.set_lr(0.0001, batch_size, train_size)
    
# load data
    print('load data')
    train_data = loadImageSet("train-images-idx3-ubyte")
    train_label = loadLabelSet("train-labels-idx1-ubyte")
    test_data = loadImageSet("t10k-images-idx3-ubyte")
    test_label = loadLabelSet("t10k-labels-idx1-ubyte")

    train_part = data_label_data(train_data, train_label)
    test_part = data_label_data(test_data, test_label)
    mnist = load_data(train_part, test_part)
#     mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  
    x_mag = tf.placeholder("float", shape=[None, 784])
    train_data = tf.reshape(x_mag, [-1,28,28,1])
    train_label = tf.placeholder("float", shape=[None, 10])
    keep_prob = tf.placeholder("float")
    num_class = 10
    print("load done")


# train
    print('train')
 
    logits = spp_net.inference(train_data, True, num_class, tp='spp')
    loss, accuracy, opt  = spp_net.train(logits, global_step, train_label)
    print('train done')
    
# evaluation
#    eval_logits = spp_net.inference(valid_data, False, num_class)
#    eval_accuracy = spp_net.loss(eval_logits, valid_label)
    with tf.Session() as sess:
        saver = tf.train.Saver(tf.global_variables())
        init = tf.global_variables_initializer()
        sess.run(init)        
        start_time = time.time()
    #    print((FLAGS.max_epochs * train_size) // batch_size)
        for step in range(max_steps):
            batch = mnist.train.next_batch(batch_size)
            opt.run( feed_dict={ x_mag:batch[0], train_label: batch[1], keep_prob: 0.5 } )
            if step % eval_frequency ==0:
                stop_time = time.time() - start_time
                start_time = time.time()
                loss_value=loss.eval(feed_dict={x_mag:batch[0], train_label: batch[1], keep_prob: 1.0})
                accu = accuracy.eval(feed_dict={x_mag:batch[0], train_label: batch[1], keep_prob: 1.0})
                print('epoch: %.2f , %.2f ms' % (step * batch_size /train_size,
                    1000 * stop_time / eval_frequency)) 
                print('train loss: ', loss_value) 
                print('train accu: ', accu)
                if step % (10*eval_frequency) ==0:
                    loss_value=loss.eval(feed_dict={x_mag:mnist.test.data, train_label: mnist.test.label, keep_prob: 1.0})
                    accu = accuracy.eval(feed_dict={x_mag:mnist.test.data, train_label: mnist.test.label, keep_prob: 1.0})
                    print('%%%%%%%%%%%%%%%%%%%%%%%test loss: ', loss_value) 
                    print('%%%%%%%%%%%%%%%%%%%%%%%test accu: ', accu)
                    


if __name__ == '__main__':
    train()




load data
load done
train
**********SPP*************
pool1.shape (?, 14, 14, 6)
conv3.shape (?, 14, 14, 16)
bin_strides [4, 7, 14]
bin_filters [5, 7, 14]
pooled_out [<tf.Tensor 'SPP/Reshape:0' shape=(?, 144) dtype=float32>, <tf.Tensor 'SPP/Reshape_1:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'SPP/Reshape_2:0' shape=(?, 16) dtype=float32>]
(?, 224)
numH 224
inference
train done
epoch: 0.00 , 1.61 ms
train loss:  0.229749
train accu:  0.04
%%%%%%%%%%%%%%%%%%%%%%%test loss:  0.230937
%%%%%%%%%%%%%%%%%%%%%%%test accu:  0.102
epoch: 1.63 , 76.42 ms
train loss:  0.224638
train accu:  0.12
epoch: 3.27 , 55.52 ms
train loss:  0.17598
train accu:  0.4
epoch: 4.90 , 53.56 ms
train loss:  0.151772
train accu:  0.5
epoch: 6.54 , 53.35 ms
train loss:  0.121772
train accu:  0.4
epoch: 8.17 , 49.73 ms
train loss:  0.119247
train accu:  0.58
epoch: 9.80 , 45.80 ms
train loss:  0.101988
train accu:  0.62
epoch: 11.44 , 44.34 ms
train loss:  0.0954563
train accu:  0.7
epoch: 13.07 , 51.44 ms
train loss

epoch: 173.20 , 50.06 ms
train loss:  0.0312496
train accu:  0.8
epoch: 174.84 , 46.49 ms
train loss:  0.0449307
train accu:  0.84
epoch: 176.47 , 47.13 ms
train loss:  0.0376778
train accu:  0.84
epoch: 178.10 , 46.94 ms
train loss:  0.0630093
train accu:  0.78
epoch: 179.74 , 43.47 ms
train loss:  0.0333831
train accu:  0.88
%%%%%%%%%%%%%%%%%%%%%%%test loss:  0.0460327
%%%%%%%%%%%%%%%%%%%%%%%test accu:  0.8196
epoch: 181.37 , 65.07 ms
train loss:  0.0437395
train accu:  0.8
epoch: 183.01 , 40.46 ms
train loss:  0.0385327
train accu:  0.84
epoch: 184.64 , 45.60 ms
train loss:  0.0433354
train accu:  0.84
epoch: 186.27 , 44.87 ms
train loss:  0.0460839
train accu:  0.88
epoch: 187.91 , 46.87 ms
train loss:  0.0250409
train accu:  0.86
epoch: 189.54 , 44.73 ms
train loss:  0.0517556
train accu:  0.86
epoch: 191.18 , 41.60 ms
train loss:  0.0462076
train accu:  0.68
epoch: 192.81 , 38.03 ms
train loss:  0.0405758
train accu:  0.82
epoch: 194.44 , 39.02 ms
train loss:  0.0356658
train acc

epoch: 354.58 , 30.34 ms
train loss:  0.0248279
train accu:  0.9
epoch: 356.21 , 29.98 ms
train loss:  0.0354309
train accu:  0.8
epoch: 357.84 , 32.06 ms
train loss:  0.0640414
train accu:  0.76
epoch: 359.48 , 28.62 ms
train loss:  0.0335344
train accu:  0.92
%%%%%%%%%%%%%%%%%%%%%%%test loss:  0.0348069
%%%%%%%%%%%%%%%%%%%%%%%test accu:  0.8664
epoch: 361.11 , 43.42 ms
train loss:  0.0352226
train accu:  0.94
epoch: 362.75 , 27.92 ms
train loss:  0.0276468
train accu:  0.86
epoch: 364.38 , 28.17 ms
train loss:  0.0415539
train accu:  0.74
epoch: 366.01 , 28.41 ms
train loss:  0.039381
train accu:  0.9
epoch: 367.65 , 27.82 ms
train loss:  0.0320748
train accu:  0.86
epoch: 369.28 , 28.51 ms
train loss:  0.0293656
train accu:  0.92
epoch: 370.92 , 27.92 ms
train loss:  0.0360549
train accu:  0.86
epoch: 372.55 , 28.27 ms
train loss:  0.0298254
train accu:  0.94
epoch: 374.18 , 28.66 ms
train loss:  0.0395097
train accu:  0.94
epoch: 375.82 , 27.62 ms
train loss:  0.0552736
train accu:

epoch: 534.31 , 29.82 ms
train loss:  0.0345038
train accu:  0.88
epoch: 535.95 , 29.66 ms
train loss:  0.0307004
train accu:  0.9
epoch: 537.58 , 29.03 ms
train loss:  0.0503599
train accu:  0.9
epoch: 539.22 , 29.04 ms
train loss:  0.023706
train accu:  0.94
%%%%%%%%%%%%%%%%%%%%%%%test loss:  0.0277985
%%%%%%%%%%%%%%%%%%%%%%%test accu:  0.8896
epoch: 540.85 , 44.22 ms
train loss:  0.025813
train accu:  0.88
epoch: 542.48 , 28.93 ms
train loss:  0.0288437
train accu:  0.8
epoch: 544.12 , 30.00 ms
train loss:  0.0293442
train accu:  0.88
epoch: 545.75 , 30.11 ms
train loss:  0.0179629
train accu:  0.88
epoch: 547.39 , 29.88 ms
train loss:  0.0216356
train accu:  0.94
